<a href="https://colab.research.google.com/github/Nio2002/-Nio2002-Python-Projects-MLOPS-DEVLOPS-AL-ML-/blob/main/celebal_Nikhilkumar(assignment_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import tensorflow as tf

In [2]:
!pip install -q efficientnet
!pip install -q plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()

Saving images.zip to images.zip


In [4]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [5]:
# Upload train and test CSV files
train_file = files.upload()
test_file = files.upload()

Saving train.csv to train.csv


Saving test.csv to test.csv


In [6]:
# Load the training and testing datasets
train_data = pd.read_csv('/content/train.csv')  # Replace with your actual train file name
test_data = pd.read_csv('/content/test.csv')    # Replace with your actual test file name

In [7]:
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [8]:
test_data.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [12]:
import zipfile
import os

# Ensure the destination directory exists.
# We will extract the zip file into a directory, not create a directory with the zip file's name.
# Let's choose a directory name for the extracted content, e.g., 'extracted_images'.
extraction_directory = '/content/extracted_images'
os.makedirs(extraction_directory, exist_ok=True)

# Assuming the uploaded file is named 'images (1).zip' based on the 'uploaded' variable
zip_file_path = '/content/images/images'

# Extract the zip file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_directory)
    print(f"Successfully extracted {zip_file_path} to {extraction_directory}")
except FileNotFoundError:
    print(f"Error: The zip file {zip_file_path} was not found.")
except zipfile.BadZipFile:
    print(f"Error: The file {zip_file_path} is not a valid zip file.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")

# You can list the contents of the extracted directory to verify
print(f"Contents of {extraction_directory}:")
print(os.listdir(extraction_directory))

An error occurred during extraction: [Errno 21] Is a directory: '/content/images/images'
Contents of /content/extracted_images:
[]


In [10]:
import zipfile

# Extract the zip file
with zipfile.ZipFile('images.zip', 'r') as zip_ref:
       zip_ref.extractall('/content/images/')

In [13]:
# Load the training and testing datasets
train_data = pd.read_csv('/content/train.csv')  # Replace with your actual train file name
test_data = pd.read_csv('/content/test.csv')    # Replace with your actual test file name

In [14]:
train_data.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [15]:
test_data.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [16]:
import cv2
import numpy as np
import os

def extract_features(image_path):
    image = cv2.imread(image_path)
    # Example: Extract color histogram
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return hist.flatten()

In [17]:
# Load the training and testing datasets
# The paths appear correct based on the provided notebook state
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

# --- Start of Added/Modified Code ---

# Create a 'label' column in the training data.
# Assuming the columns 'healthy', 'multiple_diseases', 'rust', 'scab'
# are one-hot encoded labels. We need to find which column has a 1.
# For simplicity, we'll map the column names to numerical labels.
# Define the label columns
label_cols = ['healthy', 'multiple_diseases', 'rust', 'scab']

# Find the index of the column with value 1 for each row
# This assumes exactly one of the label columns is 1 for each training image
train_data['label'] = train_data[label_cols].idxmax(axis=1)

# Optional: Map string labels to numerical labels if needed later for modeling
# Example mapping:
# label_map = {'healthy': 0, 'multiple_diseases': 1, 'rust': 2, 'scab': 3}
# train_data['numerical_label'] = train_data['label'].map(label_map)

# --- End of Added/Modified Code ---


# Keep the head calls to inspect the dataframes after modifications
print("Train data after adding 'label' column:")
print(train_data.head())

print("\nTest data head:")
print(test_data.head())

Train data after adding 'label' column:
  image_id  healthy  multiple_diseases  rust  scab              label
0  Train_0        0                  0     0     1               scab
1  Train_1        0                  1     0     0  multiple_diseases
2  Train_2        1                  0     0     0            healthy
3  Train_3        0                  0     1     0               rust
4  Train_4        1                  0     0     0            healthy

Test data head:
  image_id
0   Test_0
1   Test_1
2   Test_2
3   Test_3
4   Test_4


In [18]:
import cv2
import numpy as np
import os
import pandas as pd # Ensure pandas is imported for train_data/test_data
from tqdm import tqdm # Import tqdm for progress bar

def extract_features(image_path):
    # Check if the file exists
    if not os.path.exists(image_path):
        print(f"Warning: Image not found at {image_path}. Skipping.")
        return None # Return None or an empty array to indicate failure

    image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Warning: Could not read image at {image_path}. Skipping.")
        return None # Return None or an empty array to indicate failure

    # Example: Extract color histogram
    # Ensure the image is not empty before calculating histogram
    if image.shape[0] > 0 and image.shape[1] > 0:
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        return hist.flatten()
    else:
        print(f"Warning: Image at {image_path} has zero dimensions. Skipping.")
        return None


# Assuming train_data and test_data DataFrames are already loaded
# Ensure they have 'image_id' and 'label' columns

# Define the base directory where images are extracted
image_base_dir = '/content/images/images' # Make sure this is the correct path

# Extract features for training data
X_train = []
y_train = []

print("Extracting features for training data...")
# Use tqdm to show progress
for index, row in tqdm(train_data.iterrows(), total=train_data.shape[0]):
    # Construct the image path. You might need to add a file extension.
    # Assuming image_id in train_data is like 'Train_0' and image files are 'Train_0.jpg'
    # Check your extracted images to confirm the naming convention and extension.
    image_filename = row['image_id'] + '.jpg' # <--- Add file extension if missing in CSV
    image_path = os.path.join(image_base_dir, image_filename)

    features = extract_features(image_path)

    if features is not None: # Only append if features were successfully extracted
        X_train.append(features)
        y_train.append(row['label']) # Adjust based on your CSV structure


X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"Successfully extracted features for {len(X_train)} training images.")

# Extract features for testing data
X_test = []
y_test = []

print("Extracting features for testing data...")
# Use tqdm to show progress
for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
    # Construct the image path. Add file extension if missing in CSV.
    image_filename = row['image_id'] + '.jpg' # <--- Add file extension if missing in CSV
    image_path = os.path.join(image_base_dir, image_filename)

    features = extract_features(image_path)

    if features is not None: # Only append if features were successfully extracted
        X_test.append(features)
        y_test.append(row['label']) # Adjust based on your CSV structure


X_test = np.array(X_test)
y_test = np.array(y_test)

print(f"Successfully extracted features for {len(X_test)} testing images.")

Extracting features for training data...


100%|██████████| 1821/1821 [00:45<00:00, 40.26it/s]


Successfully extracted features for 1821 training images.
Extracting features for testing data...


  0%|          | 0/1821 [00:00<?, ?it/s]


KeyError: 'label'

In [19]:
import cv2
import numpy as np
import os
import pandas as pd # Ensure pandas is imported for train_data/test_data
from tqdm import tqdm # Import tqdm for progress bar

def extract_features(image_path):
    # Check if the file exists
    if not os.path.exists(image_path):
        print(f"Warning: Image not found at {image_path}. Skipping.")
        return None # Return None or an empty array to indicate failure

    image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Warning: Could not read image at {image_path}. Skipping.")
        return None # Return None or an empty array to indicate failure

    # Example: Extract color histogram
    # Ensure the image is not empty before calculating histogram
    if image.shape[0] > 0 and image.shape[1] > 0:
        hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        return hist.flatten()
    else:
        print(f"Warning: Image at {image_path} has zero dimensions. Skipping.")
        return None


# Assuming train_data and test_data DataFrames are already loaded
# Ensure they have 'image_id' and 'label' columns

# Define the base directory where images are extracted
image_base_dir = '/content/images/images' # Make sure this is the correct path

# Extract features for training data
X_train = []
y_train = []

print("Extracting features for training data...")
# Use tqdm to show progress
for index, row in tqdm(train_data.iterrows(), total=train_data.shape[0]):
    # Construct the image path. You might need to add a file extension.
    # Assuming image_id in train_data is like 'Train_0' and image files are 'Train_0.jpg'
    # Check your extracted images to confirm the naming convention and extension.
    image_filename = row['image_id'] + '.jpg' # <--- Add file extension if missing in CSV
    image_path = os.path.join(image_base_dir, image_filename)

    features = extract_features(image_path)

    if features is not None: # Only append if features were successfully extracted
        X_train.append(features)
        y_train.append(row['label']) # Adjust based on your CSV structure


X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"Successfully extracted features for {len(X_train)} training images.")

# Extract features for testing data
X_test = []
# y_test = [] # Remove this line as you don't have test labels

print("Extracting features for testing data...")
# Use tqdm to show progress
for index, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
    # Construct the image path. Add file extension if missing in CSV.
    image_filename = row['image_id'] + '.jpg' # <--- Add file extension if missing in CSV
    image_path = os.path.join(image_base_dir, image_filename)

    features = extract_features(image_path)

    if features is not None: # Only append if features were successfully extracted
        X_test.append(features)
        # y_test.append(row['label']) # Remove this line


X_test = np.array(X_test)
# y_test = np.array(y_test) # Remove this line


print(f"Successfully extracted features for {len(X_test)} testing images.")

Extracting features for training data...


100%|██████████| 1821/1821 [00:45<00:00, 39.84it/s]


Successfully extracted features for 1821 training images.
Extracting features for testing data...


100%|██████████| 1821/1821 [00:46<00:00, 38.75it/s]

Successfully extracted features for 1821 testing images.


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [21]:
svm_model = SVC(kernel='linear', probability=True)

In [22]:
from sklearn.linear_model import SGDClassifier
svm_model = SGDClassifier(loss='hinge')  # SVM with hinge loss
svm_model.fit(X_train, y_train)

SGDClassifier()

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)  # Adjust number of components as needed
X_train_reduced = pca.fit_transform(X_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}

# Create a GridSearchCV object
grid_search_svm = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search_svm.fit(X_train, y_train)

# Get the best parameters
best_params_svm = grid_search_svm.best_params_
print("Best SVM Parameters:", best_params_svm)
